In [74]:
%matplotlib inline

import sys
from pathlib import Path

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

SCRIPT_DIR = Path.cwd()
sys.path.append(str(SCRIPT_DIR.parent))

from util_data.dataIterator import DataIterator

Loading the processed data

In [75]:
training_data = np.load(open("training_data/mushrooms_training_data.npy", 'rb'))
training_labels = np.load(open("training_data/mushrooms_training_labels.npy", 'rb'))
validation_data = np.load(open("training_data/mushrooms_validation_data.npy", 'rb'))
validation_labels = np.load(open("training_data/mushrooms_validation_labels.npy", 'rb'))
test_data = np.load(open("training_data/mushrooms_test_data.npy", 'rb'))
test_labels = np.load(open("training_data/mushrooms_test_labels.npy", 'rb'))

print data types and shapes

In [76]:
print("Loaded data type = {}".format(type(training_data)))
print("Training data shape = {}".format(training_data.shape))
print("Training labels shape = {}".format(training_labels.shape))
print("Validation data shape = {}".format(validation_data.shape))
print("Validation labels shape = {}".format(validation_labels.shape))
print("Test data shape = {}".format(test_data.shape))
print("Test labels shape = {}".format(test_labels.shape))

Loaded data type = <class 'numpy.ndarray'>
Training data shape = (6500, 117)
Training labels shape = (6500, 1)
Validation data shape = (812, 117)
Validation labels shape = (812, 1)
Test data shape = (812, 117)
Test labels shape = (812, 1)


In [77]:
#parameters

fully_connected_layers=3
fully_connected_size=50
fully_connected_activationfunction=tf.nn.relu

In [88]:
#placeholders data input
tf.compat.v1.disable_eager_execution() 
x=tf.compat.v1.placeholder(dtype=tf.float32,shape=[None,training_data.shape[1]],name="inputs")

y=tf.compat.v1.placeholder(dtype=tf.float32,shape=[None, 1], name="labels")

learning_rate=tf.compat.v1.placeholder(dtype=tf.float32, shape=[], name="learning_rate")

global_step=tf.compat.v1.get_variable("global_step",shape=[],dtype=tf.int32,initializer=tf.constant_initializer(0),trainable=False)


ValueError: Variable global_step already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

In [89]:
print(global_step)

<tf.Variable 'hj:0' shape=() dtype=int32>
